# NOTEBOOK INI ADALAH BAGIAN DIMANA DATASET BERUPA GAMBAR AKAN DIEXPORT MENJADI FILE H5PY

catatan awal:

> untuk file .h5
*     header untuk data 'X' : "x_images"
*     header untuk data 'y' : "y_labels"

> kode label
0. ceplok
1. kawung
2. lereng
3. nitik
4. parang
5. semen
6. lung-lungan

In [6]:
import h5py
import numpy as np
from PIL import Image
import os, os.path, shutil 
import random
from sklearn.utils import shuffle
import imageio

# Functions Definition

In [7]:
def download_all_images(folder_dir):
    """ download images from many folders,
        then save image alongside with its label
        
        Parameters:
        ---------------
        folder_dir        String, file location that contain many folders

        Returns:
        ----------
        x_images_array   (N, 224, 224, 3) to be stored, images array, 
        y_images_array    int label code (N, code), associated label
    """
    counter = 0
    x_images_array = []
    y_images_array = []
    for folder_image in os.listdir(folder_dir):
        for filename in os.listdir(f"{folder_dir+folder_image}"):
            img = Image.open(f"{folder_dir+folder_image}/{filename}")
            img = img.convert('RGB')
            img = img.resize((224,224))
            arr_img = np.array(img)
            x_images_array.append(arr_img)
            y_images_array.append(folder_image)
            print(f"working on {folder_image}/{filename}: {np.shape(arr_img)}")
            counter = counter + 1

    print("Done. retrieve " + str(counter) + " datas")
    return x_images_array, y_images_array

In [8]:
def store_many_hdf5(file_location, images, labels):
    """ Stores an array of images to HDF5.
        Parameters:
        ---------------
        file_location     path to HDF5 file, string
        images           images array, (N, 32, 32, 3) to be stored
        labels           labels array, (N, 1) to be stored
    """
    num_images = len(images)

    # Create a new HDF5 file
    file = h5py.File(f"{file_location}.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "x_images", np.shape(images), h5py.h5t.STD_U8BE, data=images
    )
    meta_set = file.create_dataset(
        "y_labels", np.shape(labels), h5py.h5t.STD_U8BE, data=labels
    )
    file.close()
    print(f"{num_images} datas has been stored to HDF5")

In [9]:
def split_train_val(origin_dir, dest_dir):
    """
        split image files into train and validation set
        
        Parameters:
        ---------------
        origin_dir --> origin directory contains whole data in one dataset\n
        dest_dir --> destination directory that will MOVE the VALIDATION DATASET
 
        Returns:
        ----------
        NONE
    """
    for folder_image in os.listdir(origin_dir):
        try: 
            os.mkdir(f"{dest_dir+folder_image}") 
        except OSError as error: 
            print(error)
            
        filenames = random.sample([x for x in os.listdir(f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\{folder_image}")], 220) 
        for filename in filenames:
            shutil.move(f"{origin_dir+folder_image}\\{filename}", f"{dest_dir+folder_image}\\{filename}")
        
        print(f"SIZE ORIGIN FOLDER {folder_image}: {str(np.shape(os.listdir(origin_dir+folder_image)))}")
        print(f"SIZE DEST FOLDER {folder_image}: {str(np.shape(os.listdir(dest_dir+folder_image)))}")

# splitting randomly image inside each folder
## 20% for validation dataset
## 80% for training dataset

In [10]:
# this is to split dataset DATASET_ALLFEATURES

"""
==============
WARNING: this code alread executed. 
to re-execute please disable comment
==============
"""
# split_train_val("C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\", "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\validation\\")


'\n==============\nWARNING: this code alread executed. \nto re-execute please disable comment\n==============\n'

# Retrieve Dataset and store to h5py

In [11]:
def create_h5_dataset(source_image, h5_destination, h5_filename):
    """
    assign images into X and y data. then save to h5 file
    
    Parameters:
    source_image -> String, image directory
    h5_destination -> String, h5 destination directory 
    h5_filename -> String, h5 filename
    
    Return:
    None
    """

    print("Working...")

    # X, y = download_all_images("C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/Training/")
    X, y = download_all_images(source_image)

    # assign y as array
    y = np.array(y).astype(int)

    # SHUFFLE
    X, y = shuffle(X, y, random_state=0)

    # print shape
    print(f"X Shape: {np.shape(X)}\ny Shape: {np.shape(y)}")

    # store dataset
    dest_location = f"{h5_destination}{h5_filename}"
    store_many_hdf5(dest_location, X, y)

    print("DONE")

In [14]:
source = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\Training2\\"
dest = "C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/h5_file/"
filename = "train_geometri_augmAllFeatures_600"
create_h5_dataset(source, dest, filename)

source = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\Testing2\\"
dest = "C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/h5_file/"
filename = "val_geometri_augmAllFeatures_100"
create_h5_dataset(source, dest, filename)

Working...
working on 0/0531-cl.jpg: (224, 224, 3)
working on 0/batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/ceplok-bintang.jpg: (224, 224, 3)
working on 0/ceplok-bligon1.jpg: (224, 224, 3)
working on 0/FL==0531-cl.jpg: (224, 224, 3)
working on 0/FL==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/FL==ceplok-bintang.jpg: (224, 224, 3)
working on 0/FL==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/FL==SC015.jpg: (224, 224, 3)
working on 0/FL==SC050.jpg: (224, 224, 3)
working on 0/FL==YC047.jpg: (224, 224, 3)
working on 0/FL==YC086.jpg: (224, 224, 3)
working on 0/FL==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/FL==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/FL==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/FL==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/FL==Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg: (224,

working on 0/MI==YT056.jpg: (224, 224, 3)
working on 0/MI==YT065.jpg: (224, 224, 3)
working on 0/MI==YT080.jpg: (224, 224, 3)
working on 0/MI==YT117.jpg: (224, 224, 3)
working on 0/MI==YT173.jpg: (224, 224, 3)
working on 0/RA==0531-cl.jpg: (224, 224, 3)
working on 0/RA==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/RA==ceplok-bintang.jpg: (224, 224, 3)
working on 0/RA==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/RA==SC015.jpg: (224, 224, 3)
working on 0/RA==SC050.jpg: (224, 224, 3)
working on 0/RA==YC047.jpg: (224, 224, 3)
working on 0/RA==YC086.jpg: (224, 224, 3)
working on 0/RA==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/RA==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/RA==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/RA==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/RA==Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan

working on 0/RO==YT117.jpg: (224, 224, 3)
working on 0/RO==YT173.jpg: (224, 224, 3)
working on 0/SC015.jpg: (224, 224, 3)
working on 0/SC050.jpg: (224, 224, 3)
working on 0/SL%0%==0531-cl.jpg: (224, 224, 3)
working on 0/SL%0%==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/SL%0%==ceplok-bintang.jpg: (224, 224, 3)
working on 0/SL%0%==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/SL%0%==SC015.jpg: (224, 224, 3)
working on 0/SL%0%==SC050.jpg: (224, 224, 3)
working on 0/SL%0%==YC047.jpg: (224, 224, 3)
working on 0/SL%0%==YC086.jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg: (224,

working on 0/SL%1%==YT056.jpg: (224, 224, 3)
working on 0/SL%1%==YT065.jpg: (224, 224, 3)
working on 0/SL%1%==YT080.jpg: (224, 224, 3)
working on 0/SL%1%==YT117.jpg: (224, 224, 3)
working on 0/SL%1%==YT173.jpg: (224, 224, 3)
working on 0/SL%2%==0531-cl.jpg: (224, 224, 3)
working on 0/SL%2%==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/SL%2%==ceplok-bintang.jpg: (224, 224, 3)
working on 0/SL%2%==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/SL%2%==SC015.jpg: (224, 224, 3)
working on 0/SL%2%==SC050.jpg: (224, 224, 3)
working on 0/SL%2%==YC047.jpg: (224, 224, 3)
working on 0/SL%2%==YC086.jpg: (224, 224, 3)
working on 0/SL%2%==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/SL%2%==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/SL%2%==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/SL%2%==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/SL%2

working on 0/SL%3%==YT080.jpg: (224, 224, 3)
working on 0/SL%3%==YT117.jpg: (224, 224, 3)
working on 0/SL%3%==YT173.jpg: (224, 224, 3)
working on 0/YC047.jpg: (224, 224, 3)
working on 0/YC086.jpg: (224, 224, 3)
working on 0/Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 0/Yogya-Cap-Ceplok-Ceplok-Sogan.jpg: (224, 224, 3)
working on 0/Yogya-Cap-Grompol kecil-Grompol-Sogan (1).jpg: (224, 224, 3)
working on 0/Yogya-Cap-Grompol-Grompol-Sogan.jpg: (224, 224, 3)
working on 0/Yogya-Cap-Jamblang--Sogan.jpg: (224, 224, 3)
working on 0/Yogya-Cap-Sawitan Pintu Retno-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/Yogya-Cap-Sidodrajat Tru

...

working on 4/Yogya-Parang-Parang-Sogan (13).jpg: (224, 224, 3)
working on 4/Yogya-Parang-Parang-Sogan (3).jpg: (224, 224, 3)
working on 4/Yogya-Parang-Parang-Sogan (4).jpg: (224, 224, 3)
working on 4/Yogya-Parang-Parang-Sogan (7).jpg: (224, 224, 3)
working on 4/Yogya-Parang-Parang-Sogan (9).jpg: (224, 224, 3)
working on 4/Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (1).jpg: (224, 224, 3)
working on 4/Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (2).jpg: (224, 224, 3)
working on 4/Yogya-Tulis-Parang Klithik Gurdo-parang,Gurdo-Sogan.jpg: (224, 224, 3)
working on 4/Yogya-Tulis-Parang Plenik Seling Sisik-Parang,Sisik,Gurdo-Sogan.jpg: (224, 224, 3)
working on 4/ZO==0068-cl.jpg: (224, 224, 3)
working on 4/ZO==0170-sample-solo-jateng-btk-s76-L410b.jpg: (224, 224, 3)
working on 4/ZO==0172-cl.jpg: (224, 224, 3)
working on 4/ZO==0180-cl.jpg: (224, 224, 3)
working on 4/ZO==0181-cl.jpg: (224, 224, 3)
working on 4/ZO==0430-cl.jpg: (224, 224, 3)
working on 4/ZO==18-Foto-Motif-Batik-Pa

working on 0/SL%1%==Solo-Cap-Parang-Ceplok Teblem.jpg: (224, 224, 3)
working on 0/SL%1%==Yogya-Cap-Purbonegoro-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/SL%1%==Yogya-Cap-Tambal-Truntum,Parang,Kawung-Sogan (2).jpg: (224, 224, 3)
working on 0/SL%1%==Yogya-Cap-Truntum-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/SL%1%==Yogya-Tulis-Keong Renteng-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/SL%1%==Yogya-Tulis-Langen Mulyo-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/SL%1%==Yogya-Tulis-Putri Solo Ukel-Truntum,Gurdo,Ukel-Sogan.jpg: (224, 224, 3)
working on 0/SL%1%==Yogya-Tulis-Ukel Ceplok Bintang-Ukel,Bintang,Ceplok-Sogan.jpg: (224, 224, 3)
working on 0/SL%2%==bbr-004-batik-yogya-kombinasi-rakitan-putri-solo.jpg: (224, 224, 3)
working on 0/SL%2%==ceplok-ganggong.jpg: (224, 224, 3)
working on 0/SL%2%==Solo-Cap-Parang-Ceplok Teblem.jpg: (224, 224, 3)
working on 0/SL%2%==Yogya-Cap-Purbonegoro-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/SL%2%==Yogya-Cap-Tambal-Truntum,Parang,Kawung-S

working on 2/FL==8agt 042.jpg: (224, 224, 3)
working on 2/FL==dfasfaew.JPG: (224, 224, 3)
working on 2/FL==IMG_2621u1.jpg: (224, 224, 3)
working on 2/FL==lereng kembang.JPG: (224, 224, 3)
working on 2/FL==Lereng Suliga.jpg: (224, 224, 3)
working on 2/FL==SC007.jpg: (224, 224, 3)
working on 2/FL==SC071.jpg: (224, 224, 3)
working on 2/FL==SC089.jpg: (224, 224, 3)
working on 2/FL==Yogya-Cap-Udan Riris--Sogan (3).jpg: (224, 224, 3)
working on 2/IMG_2621u1.jpg: (224, 224, 3)
working on 2/lereng kembang.JPG: (224, 224, 3)
working on 2/Lereng Suliga.jpg: (224, 224, 3)
working on 2/MI==0166-cl.jpg: (224, 224, 3)
working on 2/MI==8agt 042.jpg: (224, 224, 3)
working on 2/MI==dfasfaew.JPG: (224, 224, 3)
working on 2/MI==IMG_2621u1.jpg: (224, 224, 3)
working on 2/MI==lereng kembang.JPG: (224, 224, 3)
working on 2/MI==Lereng Suliga.jpg: (224, 224, 3)
working on 2/MI==SC007.jpg: (224, 224, 3)
working on 2/MI==SC071.jpg: (224, 224, 3)
working on 2/MI==SC089.jpg: (224, 224, 3)
working on 2/MI==Yogya-C

working on 3/SL%2%==NITIK KETONGKENG.jpg: (224, 224, 3)
working on 3/SL%2%==Yogya-Cap-Grompol-Grompol-Sogan.jpg: (224, 224, 3)
working on 3/SL%2%==Yogya-Nitik-Nitik-Sogan (1).jpg: (224, 224, 3)
working on 3/SL%2%==Yogya-Nitik-Nitik-Sogan (14).jpg: (224, 224, 3)
working on 3/SL%2%==Yogya-Nitik-Nitik-Sogan (15).jpg: (224, 224, 3)
working on 3/SL%2%==Yogya-Nitik-Nitik-Sogan (8).jpg: (224, 224, 3)
working on 3/SL%2%==Yogya-Tulis-Nitik Kembang Jeruk-nitik-Sogan.jpg: (224, 224, 3)
working on 3/SL%2%==YT014.jpg: (224, 224, 3)
working on 3/SL%3%==0163-sample-solo-jateng-btk-s76-L410b.jpg: (224, 224, 3)
working on 3/SL%3%==3motif-nitik.jpg: (224, 224, 3)
working on 3/SL%3%==NITIK KETONGKENG.jpg: (224, 224, 3)
working on 3/SL%3%==Yogya-Cap-Grompol-Grompol-Sogan.jpg: (224, 224, 3)
working on 3/SL%3%==Yogya-Nitik-Nitik-Sogan (1).jpg: (224, 224, 3)
working on 3/SL%3%==Yogya-Nitik-Nitik-Sogan (14).jpg: (224, 224, 3)
working on 3/SL%3%==Yogya-Nitik-Nitik-Sogan (15).jpg: (224, 224, 3)
working on 3/SL%

y Shape: (500,)
500 datas has been stored to HDF5
DONE


# Retrieve Dataset

In [ ]:
print("Working...")

X, y = download_all_images("C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/validation/")

# assign y as array
y = np.array(y).astype(int)

# SHUFFLE
X, y = shuffle(X, y, random_state=0)

# print shape
print(f"X Shape: {np.shape(X)}\ny Shape: {np.shape(y)}")



# Store to HDF5

In [8]:
dest_location = "C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/h5_file/ALLFEATURES_NONSPLIT"
store_many_hdf5(dest_location, X, y)

print("DONE")

stored to HDF5
DONE
